# Part 2 - [MeanShift](http://www.chioka.in/meanshift-algorithm-for-the-rest-of-us-python) & [Camshift](https://medium.com/@claudio.vindimian/understanding-and-implementing-the-camshift-object-tracking-algorithm-python-81587c24eda8) Tracking

- First read the article about [MeanShift Algorithm](http://www.chioka.in/meanshift-algorithm-for-the-rest-of-us-python).
- MeanShift won't always detect what may be more "resonable".
- It may have been more reasonable to detect 4 clusters in the previous situation.
- MeanShift can be given a target to track, calculate the color histogram of the target area, and then keep sliding the tracking window to the closest match (the cluster center).
- Just using MeanShift won't change the window size if the target moves away or towards the camera.
- We can use [CAMShift](https://medium.com/@claudio.vindimian/understanding-and-implementing-the-camshift-object-tracking-algorithm-python-81587c24eda8) to update the size of the window.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

face_cascade = cv2.CascadeClassifier("../DATA/haarcascades/haarcascade_frontalface_default.xml")
face_rects   = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)

roi = frame[face_y : face_y+h, face_x : face_x+w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi], channels=[0], mask=None, histSize=[180], ranges=[0, 180])

cv2.normalize(roi_hist, dst=roi_hist, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        dst = cv2.calcBackProject(images=[hsv], channels=[0], hist=roi_hist, ranges=[0, 180], scale=1)
        
        ####################################################################
        ### MeanShift ###
        # ret, track_window = cv2.meanShift(probImage=dst, window=track_window, criteria=term_crit)
        
        # x, y, w, h = track_window
        # img2 = cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 0, 255), thickness=3)
        
        ### CAMShift ###
        ret, track_window = cv2.CamShift(probImage=dst, window=track_window, criteria=term_crit)
        
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, pts=[pts], isClosed=True, color=(0, 0, 255), thickness=3)
        
        ####################################################################
        
        cv2.imshow("img", img2)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()